<td>
   <a target="_blank" href="https://labelbox.com" ><img src="https://labelbox.com/blog/content/images/2021/02/logo-v4.svg" width=256/></a>
</td>

<td>
<a href="https://colab.research.google.com/github/Labelbox/labelbox-python/blob/develop/examples/model_assisted_labeling/video_mal.ipynb" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
</td>

<td>

<a href="https://github.com/Labelbox/labelbox-python/tree/develop/examples/model_assisted_labeling/video_mal.ipynb" target="_blank"><img
src="https://img.shields.io/badge/GitHub-100000?logo=github&logoColor=white" alt="GitHub"></a>
</td> 

# Video MAL

* Annotations must be created and uploaded using NDJSON
* Supported annotations that can be uploaded through the SDK:
    * Bounding box
    * Point
    * Polyline 
    * Radio classifications 
    * Checklist classifications 
* **NOT** supported:
    * Polygons 
    * Segmentation masks
    * Free form text classifications
    * Any nested classifications (under either a tool or classification) 

Please note that this list of unsupported annotations only refers to limitations for importing annotations. For example, when using the Labelbox editor, segmentation masks can be created and edited on video assets.

### Setup

In [ ]:
!pip install -q 'labelbox[data]'

In [2]:
import uuid
from labelbox import Client, LabelingFrontend, MediaType, MALPredictionImport, LabelImport
from labelbox.schema.ontology import OntologyBuilder, Tool, Classification, Option
from labelbox.schema.queue_mode import QueueMode
from labelbox.data.annotation_types import (
    Label, ObjectAnnotation,
    Rectangle, Point, Line, Radio, Checklist, ClassificationAnnotation, ClassificationAnswer
)

### Replace with your API key 
Guides on [Create an API key](https://docs.labelbox.com/docs/create-an-api-key)

In [3]:
# Add your api key
API_KEY = None
client = Client(api_key=API_KEY)

## Support annotations for video
Only NDJSON annotations are supported with video assets

### Supported NDJSON annotations

In [4]:
######## Bounding box  ###########

# NDJSON
bbox_annotation_ndjson = {
    "name" : "bbox_video",
    "segments" : [{
        "keyframes" : [
            {
              "frame": 13,
              "bbox" : {
                "top": 146.0,
                "left": 98.0,
                "height": 382.0,
                "width": 341.0
              }  
           },
           {
              "frame": 14,
              "bbox" : {
                "top": 146.0,
                "left": 98.0,
                "height": 382.0,
                "width": 341.0
              }  
           },
           {
              "frame": 15,
              "bbox" : {
                "top": 146.0,
                "left": 98.0,
                "height": 382.0,
                "width": 341.0
              }  
           }
        ]
      }
    ]
}

In [5]:
######## Point ########

#NDJSON
point_annotation_ndjson = {
    "name": "point_video", 
    "segments": [{
        "keyframes": [{
            "frame": 17,
            "point" : {
                "x": 660.134 ,
                "y": 407.926
            }
        }]
    }] 
}

In [6]:
######## Polyline ########

# NDJSON (frame based annotations are supported with NDJSON format)
polyline_frame_annotation_ndjson = {
  "name": "line_video_frame", 
  "segments": [
      {
        "keyframes": [
          {
            "frame": 5,
            "line": [{
              "x": 680,
              "y": 100
            },{
              "x": 100,
              "y": 190
            },{
              "x": 190,
              "y": 220
            }]
          },
          {
            "frame": 12,
            "line": [{
              "x": 680,
              "y": 280
            },{
              "x": 300,
              "y": 380
            },{
              "x": 400,
              "y": 460
            }]
          },
          {
            "frame": 20,
            "line": [{
              "x": 680,
              "y": 180
            },{
              "x": 100,
              "y": 200
            },{
              "x": 200,
              "y": 260
            }]
          }
        ]
      },
      {
        "keyframes": [
          {
            "frame": 24,
            "line": [{
              "x": 300,
              "y": 310
            },{
              "x": 330,
              "y": 430
            }]
          },
          {
            "frame": 45,
            "line": [{
              "x": 600,
              "y": 810
            },{
              "x": 900,
              "y": 930
            }]
          }
        ]
      }
    ]
}

In [7]:
######## Checklist classification ########

## NDJSON

## frame specific
frame_checklist_classification_ndjson = {
    "name": "checklist_class", 
    "answer": [
        { "name": "first_checklist_answer" , "frames": [{"start": 29, "end": 35 }, {"start": 48, "end": 65}]},
        { "name": "second_checklist_answer", "frames": [{"start": 29, "end": 35 }, {"start": 48, "end": 65}]} 
  ]      
}

# Global 
global_radio_classification_ndjson = {
    "name": "radio_class_global", 
    "answer": { "name": "first_radio_answer" }
}



## Upload Annotations - putting it all together

### Step 1: Import data rows into Catalog

In [8]:
from labelbox.data.annotation_types.collection import uuid4
client = Client(API_KEY)

asset = {
    "row_data": "https://storage.googleapis.com/labelbox-datasets/video-sample-data/sample-video-2.mp4", 
    "global_key": str(uuid.uuid4()),
    "media_type": "VIDEO"
}

dataset = client.create_dataset(name="video_demo_dataset")
data_row = dataset.create_data_row(asset)
print(data_row.uid)
print(data_row)

clbweokzu0hup07znaag28k3r
<DataRow {
    "created_at": "2022-12-20 15:53:50+00:00",
    "external_id": null,
    "global_key": "3321ae25-3533-4c93-89ea-2cd0cb6f5acb",
    "media_attributes": {},
    "metadata": [],
    "metadata_fields": [],
    "row_data": "https://storage.googleapis.com/labelbox-datasets/video-sample-data/sample-video-2.mp4",
    "uid": "clbweokzu0hup07znaag28k3r",
    "updated_at": "2022-12-20 15:53:50+00:00"
}>


### Step 2: Create/select an ontology
Your project should have the correct ontology setup with all the tools and classifications supported for your annotations, and the tool names and classification instructions should match the `name`/`instructions` fields in your annotations to ensure the correct feature schemas are matched.

For example, when we create the bounding box annotation [above](https://colab.research.google.com/drive/1S3HBa10jwfFFNB71Uid-crUDdhfEGnXh#scrollTo=suburban-crowd&line=10&uniqifier=1), we provided the `name` as `bbox_video`. Now, when we setup our ontology, we must ensure that the name of my bounding box tool is also `bbox_video`. The same alignment must hold true for the other tools and classifications we create in our ontology.


[Documentation for reference ](https://docs.labelbox.com/reference/import-text-annotations)

In [9]:
ontology_builder = OntologyBuilder(
    tools=[
        Tool(tool=Tool.Type.BBOX, name="bbox_video"),
        Tool(tool=Tool.Type.POINT, name="point_video"),
        Tool(tool=Tool.Type.LINE, name="line_video_frame")
    ],
    classifications=[ 
        Classification(
            class_type=Classification.Type.CHECKLIST, 
            instructions="checklist_class",
            scope = Classification.Scope.INDEX, ## Need to defined scope for frame classifications 
            options=[ 
                Option(value="first_checklist_answer"),
                Option(value="second_checklist_answer")
            ]
        ),
        Classification(
            class_type=Classification.Type.RADIO, 
            instructions="radio_class_global",
            options=[ 
                Option(value="first_radio_answer"),
                Option(value="second_radio_answer")
            ]
        ) 
    ]  
)

ontology = client.create_ontology("Ontology Video Annotations", ontology_builder.asdict())

### Step 3: Create a labeling project 
Connect the ontology to the labeling project.

In [10]:
# Project defaults to batch mode with benchmark quality settings if this argument is not provided
# Queue mode will be deprecated once dataset mode is deprecated

project = client.create_project(name="video_project_demo",
                                    queue_mode=QueueMode.Batch,
                                    media_type=MediaType.Video)

## connect ontology to your project
project.setup_editor(ontology)

######################### DATASET CONSENSUS OPTION ########################
# Note that dataset base projects will be deprecated in the near future.

# To use Datasets/Consensus instead of Batches/Benchmarks use the following query: 
# In this case, 10% of all data rows need to be annotated by three labelers.

# dataset_project = client.create_project(name="dataset-test-project",
#                                 description="a description",
#                                 media_type=MediaType.Text,
#                                 auto_audit_percentage=0.1,
#                                 auto_audit_number_of_labels=3,
#                                 queue_mode=QueueMode.Dataset)

# dataset_project.datasets.connect(dataset)

### Step 4: Send a batch of data rows to the project

In [16]:
# Create batches

# Create a batch to send to your MAL project
batch = project.create_batch(
  "first-batch-video-demo2", # Each batch in a project must have a unique name
  dataset.export_data_rows(), # A paginated collection of data row objects
  5 # priority between 1(Highest) - 5(lowest)
)

print("Batch: ", batch)

Batch:  <Batch {
    "consensus_settings_json": "{\"numberOfLabels\":1,\"coveragePercentage\":0}",
    "created_at": "2022-12-20 15:57:26+00:00",
    "name": "first-batch-video-demo2",
    "size": 1,
    "uid": "00165960-807f-11ed-bf7f-51b4711aa983",
    "updated_at": "2022-12-20 15:57:26+00:00"
}>


### Step 5: Create the annotations payload 
Create the annotations payload using the snippets of code above.

Labelbox supports two formats for the annotations payload: NDJSON and Python Annotation types. However, for video assets, only NDJSON format is supported.

#### NDJSON annotations
Here we create the complete `label_ndjson` payload of annotations. There is one annotation for each *reference to an annotation* that we created [above](https://colab.research.google.com/drive/1S3HBa10jwfFFNB71Uid-crUDdhfEGnXh#scrollTo=kLT9P-WYk4Nr&line=1&uniqifier=1).

In [12]:
label_ndjson = []

for annotations in [point_annotation_ndjson,
                    bbox_annotation_ndjson,
                    polyline_frame_annotation_ndjson, 
                    frame_checklist_classification_ndjson, 
                    global_radio_classification_ndjson
                    ]:      
  annotations.update({
      'uuid' : str(uuid.uuid4()),
      'dataRow': {
          'id':  next(dataset.export_data_rows()).uid
      }
  })
  label_ndjson.append(annotations)


In [14]:
  label_ndjson

[{'name': 'point_video',
  'segments': [{'keyframes': [{'frame': 17,
      'point': {'x': 660.134, 'y': 407.926}}]}],
  'uuid': '75cb8bb5-b844-4b16-97b9-be6c0e92bc85',
  'dataRow': {'id': 'clbweokzu0hup07znaag28k3r'}},
 {'name': 'bbox_video',
  'segments': [{'keyframes': [{'frame': 13,
      'bbox': {'top': 146.0, 'left': 98.0, 'height': 382.0, 'width': 341.0}},
     {'frame': 14,
      'bbox': {'top': 146.0, 'left': 98.0, 'height': 382.0, 'width': 341.0}},
     {'frame': 15,
      'bbox': {'top': 146.0,
       'left': 98.0,
       'height': 382.0,
       'width': 341.0}}]}],
  'uuid': '36fb5752-177a-46bc-9531-240eb402877d',
  'dataRow': {'id': 'clbweokzu0hup07znaag28k3r'}},
 {'name': 'line_video_frame',
  'segments': [{'keyframes': [{'frame': 5,
      'line': [{'x': 680, 'y': 100},
       {'x': 100, 'y': 190},
       {'x': 190, 'y': 220}]},
     {'frame': 12,
      'line': [{'x': 680, 'y': 280},
       {'x': 300, 'y': 380},
       {'x': 400, 'y': 460}]},
     {'frame': 20,
      'line

### Step 6: Upload annotations to a project as pre-labels or completed labels
For the purpose of this tutorial only run one of the label imports at once, otherwise the previous import might get overwritten.

#### Model-Assisted Labeling (MAL)

In [13]:
# Upload MAL label for this data row in project
upload_job_mal = MALPredictionImport.create_from_objects(
    client = client, 
    project_id = project.uid, 
    name="mal_import_job-" + str(uuid.uuid4()), 
    predictions=label_ndjson)

upload_job_mal.wait_until_done();
print("Errors:", upload_job_mal.errors)
print("   ")

Errors: []
   


#### Label Import

In [17]:
upload_job_label_import = LabelImport.create_from_objects(
    client = client,
    project_id = project.uid, 
    name = "label_import_job-" + str(uuid.uuid4()),
    labels=label_ndjson
)

upload_job_label_import.wait_until_done();
print("Errors:", upload_job_label_import.errors)
print("   ")

Errors: []
   


### Optional deletions for cleanup

In [ ]:
# Delete Project
# project.delete()
# dataset.delete()